# Увод у вероватноћу и статистику
У овом нотебуку ћемо се играти са неким од концепата које смо раније дискутовали. Многи концепти из вероватноће и статистике су добро представљени у главним библиотекама за обраду података у Python-у, као што су `numpy` и `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Случајне променљиве и расподеле
Хајде да почнемо са цртањем узорка од 30 вредности из једнолике расподеле од 0 до 9. Такође ћемо израчунати средњу вредност и варијансу.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Да бисмо визуелно проценили колико различитих вредности има у узорку, можемо нацртати **хистограм**:


In [ ]:
plt.hist(sample)
plt.show()

## Анализа стварних података

Средња вредност и дисперзија су веома важни приликом анализе стварних светских података. Хајде да учитамо податке о бејзбол играчима са [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Ми користимо пакет зван [**Pandas**](https://pandas.pydata.org/) за анализу података. Више ћемо говорити о Pandas-у и раду са подацима у Питону касније у овом курсу.

Израчунaјмо просечне вредности за старост, висину и тежину:


In [ ]:
df[['Age','Height','Weight']].mean()

Сада хајде да се фокусирамо на висину и израчунамо стандардну девијацију и варијансу:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Поред средње вредности, има смисла погледати и медијану и квартиле. Они се могу визуализовати помоћу **бокс плота**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Такође можемо направити кутијасте дијаграме за подскупове нашег скупа података, на пример, груписане по улози играча.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Напомена**: Овај дијаграм сугерише да су просечне висине првих базмена веће од висина других базмена. Касније ћемо научити како можемо формалније тестирати ову хипотезу и како показати да су наши подаци статистички значајни за то.

Године, висина и тежина су све континуиране случајне променљиве. Шта мислите какав им је распоред? Добар начин да то сазнате је да исцртате хистограм вредности:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Нормална расподела

Хајде да креирамо вештачки узорак тежина који следи нормалну расподелу са истим средњом вредношћу и варијансом као и наши стварни подаци:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Пошто су већина вредности у стварном животу нормално распоређене, не бисмо требали користити униформни генератор случајних бројева за генерисање узорка података. Ево шта се дешава ако покушамо да генеришемо тежине са униформном дистрибуцијом (генерисано помоћу `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Интервали поуздања

Хајде сада да израчунамо интервале поуздања за тежине и висине бејзбол играча. Користићемо код [са ове stackoverflow расправе](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Тестирање хипотеза

Хајде да истражимо различите улоге у нашем скупу података о бејзбол играчима:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Хајде да тестирамо хипотезу да су играчи прве базе виши од играча друге базе. Најједноставнији начин да то урадимо је да тестирамо интервале поузданости:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Можемо видети да се интервали не преклапају.

Статистички исправнији начин да се докаже хипотеза је коришћењем **Student t-теста**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Две вредности које враћа функција `ttest_ind` су:
* p-вредност се може сматрати вероватноћом да две дистрибуције имају исти просек. У нашем случају, она је врло ниска, што значи да постоје чврсти докази који подржавају да су први базични играчи виши.
* t-вредност је посредна вредност нормализоване разлике просека која се користи у t-тесту, и упоређује се са праговном вредношћу за дати ниво поверења.


## Симулација нормалне расподеле помоћу централне граничне теореме

Псеудо-рандом генератор у Пајтону је дизајниран да нам да једноличну расподелу. Ако желимо да креирамо генератор за нормалну расподелу, можемо користити централну граничну теорему. Да бисмо добили нормално распоређену вредност, једноставно ћемо израчунати средњу вредност узорка добијеног једноличним генератором.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Корелација и Зла бејзбол корпорација

Корелација нам омогућава да пронађемо везе између секвенци података. У нашем примеру за игру, претпоставимо да постоји зла бејзбол корпорација која плаћа своје играче у зависности од њихове висине - што је играч виши, то више новца добија. Претпоставимо да постоји базична плата од 1000 долара и додатни бонус од 0 до 100 долара, у зависности од висине. Узећемо праве играче из МЛБ-а и израчунати њихове замишљене плате:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Хајде сада да израчунамо коваринацију и корелацију тих секвенци. `np.cov` ће нам дати такозвану **матрицу коваринације**, која представља проширење коваринације на више променљивих. Елемент $M_{ij}$ матрице коваринације $M$ је корелација између улазних променљивих $X_i$ и $X_j$, а дијагоналне вредности $M_{ii}$ представљају дисперзију $X_i$. Слично томе, `np.corrcoef` ће нам дати **матрицу корелације**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Корелација једнака 1 значи да постоји јака **линеарна веза** између две променљиве. Можемо визуелно видети линеарну везу тако што ћемо нацртати једну вредност у односу на другу:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Хајде да видимо шта се дешава ако релација није линеарна. Претпоставимо да је наша корпорација одлучила да сакрије очигледну линеарну зависност између висина и плата, и увела неку нелинеарност у формулу, као што је `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

У овом случају, корелација је нешто мања, али је и даље прилично висока. Сада, да бисмо везу учинили још мање очигледном, можда ћемо хтети да додамо неку додатну случајност додавањем неке случајне променљиве на плату. Хајде да видимо шта се дешава:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Можете ли да погодите зашто се тачке поређају у вертикалне линије овако?

Посматрали смо корелацију између вештачки конструисаног концепта као што је зарада и посматране варијабле *висина*. Хајде да видимо да ли се две посматране варијабле, као што су висина и тежина, такође корелирају:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Нажалост, нисмо добили никакве резултате - само неке чудне `nan` вредности. То је због тога што су неке вредности у нашој серији неодређене, представљене као `nan`, што узрокује да и резултат операције буде неодређен. Гледајући матрицу можемо видети да је `Weight` проблематичан колона, зато што је израчуната аутокорелација између вредности `Height`.

> Овај пример показује значај **припреме података** и **чишћења**. Без одговарајућих података не можемо ништа израчунати.

Хајде да користимо методу `fillna` да попунимо недостајуће вредности и израчунамо корелацију: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Постоји заиста корелација, али не толико јака као у нашем вештачком примеру. Заиста, ако погледамо дијаграм распршења једне вредности у односу на другу, веза би била знатно мање очигледна:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Закључак

У овом бележнику смо научили како изводити основне операције на подацима ради израчунавања статистичких функција. Сада знамо како користити комплетан скуп математичких и статистичких алата да бисмо доказали неке хипотезе и како обрачунати интервале поузданости за произвољне променљиве узевши узорак података.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Одрицање од одговорности**:
Овај документ је преведен коришћењем AI сервиса за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Док се трудимо да обезбедимо прецизност, молимо имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитетним извором. За критичне информације препоручује се стручни људски превод. Нисмо одговорни за било каква неспоразумевања или погрешне тумачења настала употребом овог превода.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
